## Productivizar modelo

In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import sklearn
import pickle
import joblib
import json
import traceback
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
app = Flask(__name__)

In [3]:
# Cargar el codificador y el modelo

with open('label_encoders1.pkl', 'rb') as file:
    label_encoders = pickle.load(file)

with open('model_xgb_optimized.pkl', 'rb') as file:
    model = pickle.load(file)


In [4]:
# Función de transformación segura
def safe_transform(encoder_dict, series, column_name):
    """Transformar categorías con un encoder, manejando valores desconocidos."""
    if column_name not in encoder_dict:
        raise ValueError(f"No hay codificador para la columna {column_name}")
    
    encoder = encoder_dict[column_name]
    # Transformar valores usando el encoder
    try:
        return series.map(lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1)
    except ValueError:
        # Para categorías desconocidas, asignar un valor por defecto 
        return series.map(lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1)

In [5]:
# Definir la ruta de predicción
@app.route('/prediccion', methods=['POST'])
def prediccion():
    try:
        data = request.get_json(force=True)
        
        # Asegúrate de que 'data' sea una lista de diccionarios o un diccionario
        if isinstance(data, dict):
            data = [data]  # Convertir un solo diccionario en una lista
        
        if not isinstance(data, list) or not all(isinstance(item, dict) for item in data):
            return jsonify({"error": "Se espera una lista de diccionarios"}), 400
        
        df = pd.json_normalize(data)

        # Renombrar las columnas si es necesario
        if 'Account' in df.columns or 'Account.1' in df.columns:
            df.rename(columns={'Account': 'Account2', 'Account.1': 'Account4'}, inplace=True)

        # Eliminar la columna "Is Laundering" si existe
        if 'Is Laundering' in df.columns:
            df = df.drop(columns=['Is Laundering'])
            
        # Asegúrate de que todas las columnas necesarias estén presentes
        required_columns = ['From Bank', 'Account2', 'To Bank', 'Account4', 
                            'Amount Received', 'Receiving Currency', 'Amount Paid', 
                            'Payment Currency', 'Payment Format']

        # Verificar y procesar la columna 'Timestamp'
        if 'Timestamp' in df.columns:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
            df['Year'] = df['Timestamp'].dt.year
            df['Month'] = df['Timestamp'].dt.month
            df['Day'] = df['Timestamp'].dt.day
            df['Hour'] = df['Timestamp'].dt.hour
            df['Minute'] = df['Timestamp'].dt.minute
            df = df.drop(columns=['Timestamp'])

        # Verifica si faltan columnas requeridas
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            return jsonify({"error": f"Faltan columnas requeridas: {', '.join(missing_columns)}"}), 400
            
        # Transformar las columnas categóricas usando el encoder cargado
        categorical_columns = ['From Bank', 'To Bank', 'Receiving Currency', 'Payment Currency', 'Payment Format', 'Account2', 'Account4']
        for col in categorical_columns:
            if col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = safe_transform(label_encoders, df[col], col)
        
        # Verificar y transformar 'Account4' si es necesario
        if 'Account4' in df.columns:
            try:
                df['Account4'] = pd.to_numeric(df['Account4'], errors='coerce')
                df['Account4'].fillna(df['Account4'].mean(), inplace=True)
            except Exception as e:
                print(f"Error al transformar 'Account4': {e}")

        # Convertir a numpy array
        numeric_df = df.select_dtypes(include=[float, int])
        data_to_numpy = numeric_df.values.astype(float)
        
        # Hacer la predicción
        prediction = model.predict(data_to_numpy)
        prediction_proba = model.predict_proba(data_to_numpy)
        
        # Convertir predicciones y probabilidades a listas para el JSON de respuesta
        prediction_list = prediction.tolist()
        prediction_proba_list = prediction_proba.tolist()
        
        return jsonify({
            "prediction": prediction_list,
            "probability": prediction_proba_list
        })
    
    except Exception as e:
        print("Error durante la predicción:")
        print(e)
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500

In [ ]:
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [16/Sep/2024 09:55:10] "POST /prediccion HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 09:55:13] "POST /prediccion HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 09:55:15] "POST /prediccion HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2024 09:55:26] "POST /prediccion HTTP/1.1" 200 -
